<a href="https://colab.research.google.com/github/duongquangvinh/Fundamental-Machine-Learning-model/blob/main/pytorch_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Modules

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
from tqdm import tqdm
import numpy as np

## Data Preprocess

In [ ]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root="./CIFAR10", train=True, transform=transform_train, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)

testset = torchvision.datasets.CIFAR10(root="./CIFAR10", train=False, transform=transform_test, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2, drop_last=True)

## Define Model

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # We will use 3 maxpool, therefore input size will decrease by a factor of 2 in 3 times (32->16->8->4)
        self.fc1 = nn.Linear(4*4*64, 500)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 4*4*64)
        x = F.relu(self.fc1(x))
        
        # Applying dropout b/t layers which exchange highest parameters. This is a good practice
        x = self.dropout1(x)
        x = self.fc2(x)
        return x

In [ ]:
net = LeNet()
net = net.to("cuda")

## Training part

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 200], gamma=0.1)

In [ ]:
best_acc = 0
for epoch in range(0,20):
    net.train()
    running_loss = 0.0
    for batch_idx, (imgs, labels) in enumerate(trainloader):
        img = imgs.to("cuda")
        label = labels.to("cuda")
        optimizer.zero_grad()
        output = net(img)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    print(f"Training loss: {running_loss/len(trainloader)}")

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (imgs, labels) in enumerate(testloader):
            img = imgs.to("cuda")
            label = labels.to("cuda")
            output = net(img)
            loss = criterion(output, label)
            test_loss += loss.item()
            _, predict = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predict == label).sum().item()
    print(f"Test loss: {test_loss/len(testloader)}")
    print(f"Accuracy: {100*correct/total}")

    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/cifar10_ckpt.pth')
        best_acc = acc

Training loss: 1.9717460505941002
Test loss: 1.9174343294058092
Accuracy: 32.44190705128205
Saving..
Training loss: 1.962307456360889
Test loss: 1.9134767980147631
Accuracy: 32.44190705128205
Training loss: 1.9557198725650313
Test loss: 1.9100650250911713
Accuracy: 32.522035256410255
Saving..
Training loss: 1.9556984822057122
Test loss: 1.9064268668492634
Accuracy: 32.58213141025641
Saving..
Training loss: 1.9492150013944403
Test loss: 1.90194848103401
Accuracy: 32.62219551282051
Saving..
Training loss: 1.9455132325693831
Test loss: 1.8987674881250431
Accuracy: 32.79246794871795
Saving..
Training loss: 1.9420583673994916
Test loss: 1.893985234774076
Accuracy: 32.93269230769231
Saving..
Training loss: 1.9403825461024968
Test loss: 1.8905585209528606
Accuracy: 33.072916666666664
Saving..
Training loss: 1.936093641762239
Test loss: 1.8871068641161308
Accuracy: 33.2431891025641
Saving..
Training loss: 1.933196950813567
Test loss: 1.8830004571340022
Accuracy: 33.483573717948715
Saving..
Tra